In [1]:
using CUDA
using DelimitedFiles
include("../../src/forces_func.jl")
include("../../src/struct_data.jl")

Aggregate

In [2]:
@make_struct_func Cubic

force_func (generic function with 1 method)

## Add Data

In [3]:
parameters = Model(
    Cubic(
        μ₁      = 0.1,
        rₘᵢₙ    = 2,
        rₘₐₓ    = 3
    ),
    Contractile(
        fₚ      = 0.0000001
    ),
    Time(
        t_f     = 100000, 
        dt      = 0.1
    ),
    Neighbor(
        n_knn   = 50,
        nn      = 12
    ),
    Geometry(
        R_agg   = 9,
        num_agg = 2
    ),
    Simulation(
        n_text  = 200,
        path    = "BBB.xyz"
    )
)

Model(Cubic(0.1, 2.0, 3.0), Contractile(1.0e-7), Time(100000.0, 0.1), Neighbor(50, 12), Geometry(9.0, 2), Simulation(200, "BBB.xyz"))

In [4]:
X = Float32.(readdlm("../../data/init/Sphere/$(Int(parameters.Geometry.R_agg)).xyz")[3:end,2:end]) |> cu
Agg = Aggregate(Neighbor(50, 12), PositionCell(X));

In [5]:
## Var before func
force(r) =  force_func(parameters.Force::Cubic, r)

force (generic function with 1 method)

## neighbor.jl -> cu_knn()

In [6]:
function cu_knn(Agg::Aggregate)
    # Definig Variables for calculing knn
    global Agg
    
    # Defining Coordinates of each cell on the aggregates
    Agg.Neighbor.i_Cell = reshape(
                repeat(
                    Agg.Position.X, 
                    size(Agg.Position.X ,1)
                ), 
                size(Agg.Position.X ,1), 
                size(Agg.Position.X ,1), 
                3
            ) - 
            reshape(
                repeat(
                    Agg.Position.X, 
                    inner=(size(Agg.Position.X ,1),1)
                ), 
                size(Agg.Position.X ,1), 
                size(Agg.Position.X ,1), 
                3
            )

    # Calculating Norm on every cell on the aggregate
    Agg.Neighbor.Dist = sqrt.(
                Agg.Neighbor.i_Cell[:,:,1] .^ 2 + 
                Agg.Neighbor.i_Cell[:,:,2] .^ 2 + 
                Agg.Neighbor.i_Cell[:,:,3] .^ 2
                )
    # # i_Cell = nothing; GC.gc(true)

    # Calculating index of knof each cell in the aggregate
    for i = 1:Agg.ParNeighbor.nn
        Agg.Neighbor.idx[i,:] = findmin(Agg.Neighbor.Dist; dims=1)[2]
        Agg.Neighbor.Dist[Agg.Neighbor.idx[i,:]] .= Inf
    end
    synchronize()
end

cu_knn (generic function with 1 method)

In [7]:
CUDA.@time cu_knn(Agg)
Agg.Neighbor.idx

LoadError: InterruptException:

## run_event.jl -> simulate() -> rand_idx

In [ ]:
Agg.Neighbor.rand_idx = getindex.(
                            Agg.Neighbor.idx, 1
                            )[getindex.(rand(
                                    2:Agg.ParNeighbor.nn,
                                    2*size(Agg.Position.X,1)
                                ),1),:]

1076×538 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 27  28  29   8  10  34  50   3   4   5  …  502  505  529  531  510  513  514
 25  26   8   5   9  32   3  37  38  39     489  529  535  530  508  511  512
  2   1   7   3   6  11  36   9  10  11     536  528  533  536  532  538  537
 37  38  31   9  31  47  35   4   5   6     504  507  530  507  496  500  501
 30  31  37  37  39   5  13  14  15  16     531  500  500  502  535  506  507
 37  38  31   9  31  47  35   4   5   6  …  504  507  530  507  496  500  501
 30  31  37  37  39   5  13  14  15  16     531  500  500  502  535  506  507
 77  78  44  30  33  87  37  51  52  53     449  493  506  509  456  460  461
 31  32   4  38   4  10   8   7   8   9     503  534  501  534  531  507  508
  3   4   1   1   2  39  44  45  46  47     497  537  537  538  502  533  534
  2   1   7   3   6  11  36   9  10  11  …  536  528  533  536  532  538  537
 30  31  37  37  39   5  13  14  15  16     531  500  500  502  535  506  507
 26  27  30  

## forces.jl -> cu_forces

In [ ]:
function cui_force(t::Time, f::Contractile, Agg::Aggregate)
    # Definig Variables for calculing dX
    global Agg

    # Calculating distance for random forces (contractile)
    Agg.Force.r_p = Agg.Position.X .- 
                        Agg.Position.X[
                            Agg.Neighbor.rand_idx[
                                Int.(mod(
                                    Agg.t, size(Agg.Position.X, 1)
                                ) .+ 1),
                            :],
                        :]
    
    # Finding Distances/Norm for random forces
    Agg.Force.dist_p = sum(Agg.Force.r_p .^ 2, dims=2).^ 0.5

    # Finding distances
    Agg.Force.r = reshape(
            repeat(Agg.Position.X, inner=(Agg.ParNeighbor.nn,1)), 
            Agg.ParNeighbor.nn, size(Agg.Position.X)[1], 3
        ) .- 
        Agg.Position.X[getindex.(Agg.Neighbor.idx,1),:]

    # Finding Distances(Norm)
    Agg.Force.dist = ((sum(Agg.Force.r .^ 2, dims=3)) .^ 0.5)[:,:,1]

    # # Finding forces for each cell
    Agg.Force.F = force(Agg.Force.dist) .* Agg.Force.r ./ Agg.Force.dist

    # # Calculating de dX   -> dX[i,:] +=  r/dist * F
    Agg.Position.dX = sum(Agg.Force.F[2:end,:,:]; dims=1)[1,:,:] -                                       
                        f.fₚ .* (Agg.Force.r_p ./ Agg.Force.dist_p)
    synchronize()
end

cui_force (generic function with 1 method)

In [ ]:
cui_force(
    Time(100000, 0.1),
    Contractile(0.00002),
    Agg
)
Agg.Position.dX

538×3 CuArray{Float64, 2, CUDA.Mem.DeviceBuffer}:
  1.41604e-5   -0.000157369   0.00365809
  1.41604e-5   -0.000157369   0.00365809
 -0.000103355   0.00148359    0.00365808
 -9.96568e-6   -0.000354997   0.00222232
  9.91603e-5    0.00150907    0.00364654
  1.41604e-5    0.00132928    0.00365808
  0.000338516  -0.000982239   0.00365986
 -0.00172613   -0.000609538   0.00224962
 -0.00172613   -0.000609538   0.00224962
 -0.00172613   -0.000609538   0.00224962
 -0.000263474  -0.000852472   0.00365982
 -0.000192505  -0.000279385   0.003533
 -0.00169617    0.000724194   0.0021228
  ⋮                          
  0.0           0.00028278   -0.00353777
  0.000338516   0.000982239  -0.00365986
 -0.00172613    0.000609538  -0.00224962
 -0.00172613    0.000609538  -0.00224962
 -0.00172613    0.000609538  -0.00224962
 -0.000263474   0.000852472  -0.00365982
 -0.000103355  -0.00148359   -0.00365808
 -9.96568e-6    0.000354997  -0.00222232
  9.91603e-5   -0.00150907   -0.00364654
  1.41604e-5   -0.001